Pablo Jimeno - UPV/EHU 2016

In [ ]:
##################################################################################################################
### Cluster class utilities:
##################################################################################################################

#-------------------------------------------------------------------
class Cluster(object):
    """Defines Cluster object."""
    def __init__(self, sample, id):
        self.id = id
        self.sample_mask = sample['id']==id
        self.ra = sample['ra'][self.sample_mask][0]
        self.dec = sample['dec'][self.sample_mask][0]
        
        self.skycoord = SkyCoord(self.ra*u.degree, self.dec*u.degree, frame='icrs')
        self.l = self.skycoord.galactic.l # l: gal longitude in deg
        self.b = self.skycoord.galactic.b # b: gal latitude in deg
        
        if 'photo_z' in sample:
            self.photo_z = sample['photo_z'][self.sample_mask][0]
        if 'spec_z' in sample:
            self.spec_z = sample['spec_z'][self.sample_mask][0]
        if 'z' in sample:
            self.z = sample['z'][self.sample_mask][0]
        if 'rich' in sample:
            self.rich = sample['rich'][self.sample_mask][0]
                
    def set_id(self, id):
        return self.id
    
    def get_ra(self):
        return self.ra
    
    def get_dec(self):
        return self.dec
    
    def get_photo_z(self):
        return self.photo_z
    
    def get_spec_z(self):
        return self.spec_z
    
    
##################################################################################################################
### Mass-Richness relation utilities:
##################################################################################################################

#-------------------------------------------------------------------
def M_from_Robs(Robs, MR_slope, MR_scatter, MR_logM0=0.):

    def M_from_R(R):
        M_val = np.exp(MR_logM0)*(R/60.)**MR_slope
        return M_val

    def prob_R_Robs(R, Robs):
        x = (np.log(R) - np.log(Robs))/np.sqrt(2.*MR_scatter**2.)
        prob = np.exp(-x**2.)/np.sqrt(2.*np.pi*MR_scatter**2.)
        return prob

    def M_from_prob(M_vector, prob_vector):
        prob_norm = integrate.trapz(prob_vector, x=M_vector)
        M_int = integrate.trapz(M_vector*prob_vector/prob_norm, x=M_vector)
        return M_int

    R_vector = np.arange(0.1, 1000.01, 0.1)
    
    prob_vector = prob_R_Robs(R_vector, Robs)
    M_vector = M_from_R(R_vector)
    M_int = M_from_prob(M_vector, prob_vector)
    
    return M_int
